# Fazendinha Contente
## Trabalho II - Computação Gráfica
### Membros do grupo:
- Beatriz Aparecida Diniz, 11925430
- João Lucas Rodrigues Constantino, 11795763
- Leandro Sena, 9293060
- Tulio Santana, 11795526

In [21]:
# Para instalar a API PyWaveFront
# Necessário para carregar os modelos
%pip install pywavefront

Note: you may need to restart the kernel to use updated packages.


# Inicialização da Janela

In [22]:
# Dependências gerais
import math

# Dependências locais
from utils.glfw_window import GLFWWindow
from utils.opengl_session import OpenGLSession
from utils.camera import RestrictedCamera
from utils.components import *
from utils.transformations import *

# Propriedades da janela
WINDOW_WIDTH = 1280
WINDOW_HEIGHT = 720
WINDOW_TITLE = "Rave nas montanhas"

# Inicialização da janela e da sessão
window = GLFWWindow(WINDOW_WIDTH, WINDOW_HEIGHT, WINDOW_TITLE)
session = OpenGLSession(window)

# Inicialização da câmera
MIN_XPOS, MIN_YPOS, MIN_ZPOS = -10.0, -10.0, -10.0
MAX_XPOS, MAX_YPOS, MAX_ZPOS = +10.0, +10.0, +10.0
camera = RestrictedCamera (
    speed = 0.2, 
    last_xpos=WINDOW_WIDTH/2, 
    last_ypos=WINDOW_HEIGHT/2, 
    min_xpos=MIN_XPOS, min_ypos=MIN_YPOS, min_zpos=MIN_ZPOS,
    max_xpos=MAX_XPOS, max_ypos=MAX_YPOS, max_zpos=MAX_ZPOS
)

# Construção dos Componentes

In [23]:
# Definição dos Objetos e deslocamentos iniciais na cena

# Modelo da Garota
girl = WaveFrontObject (
    "obj/girl/girl OBJ.obj"
)
girl.set_model(scale(0.6, 0.6, 0.6, girl.model))
girl.set_model(translate(0.6, 10.8, 0.1, girl.model))

# Modelo da Torre
tower = WaveFrontObject("obj/tower/wooden watch tower2.obj")
tower.set_model(scale(3.0, 1.0, 1.0, tower.model))

# Modelo da Lua
moon = WaveFrontObject("obj/moon/Moon 2K.obj")
moon.set_model(translate(10.0, 20.0, 2.0, moon.model))

# Modelo da Mesa
table = WaveFrontObject("obj/table/table.obj")
table.set_model(scale(0.04, 0.04, 0.07, table.model))
table.set_model(translate(-52.0, 161.75, 1.0, table.model))

# Modelo do Penguin
penguin = WaveFrontObject("obj/penguin/PenguinBaseMesh.obj")
penguin.set_model(scale(2.0, 2.0, 2.0, penguin.model))
penguin.set_model(translate(2.0, 0.25, 2.0, penguin.model))

# Modelo do Gerador
generator = WaveFrontObject("obj/generator/Emergency_Backup_Generator-(Wavefront OBJ).obj")
generator.set_model(translate(-7.0, 2.50, 6.0, generator.model))

# Modelo da Planta
plant = WaveFrontObject("obj/plant/Low-Poly Plant_.obj")
plant.set_model(translate(-3.0, 6.5, -0.75, plant.model))

# Modelo do Servidor
server = WaveFrontObject("obj/server/server computer.obj")
server.set_model(scale(0.005, 0.005, 0.005, server.model))
server.set_model(translate(1.0, 13.0, 10.0, server.model))

# Registro das componentes na sessão
session.add_components([girl, tower, moon, table, penguin, generator, plant, server])

Unimplemented OBJ format statement 's' on line 's 1'


Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's off'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'


In [24]:
# Para testes
'''
from pywavefront import Wavefront
scene = Wavefront("obj/tower/wooden watch tower2.obj")
for name, material in scene.materials.items():
    print(material.texture._path)
'''

Unimplemented OBJ format statement 's' on line 's 1'


# Eventos de teclado e de mouse

In [26]:
# Para aumentar ou diminuir a velocidade da câmera
def modify_camera_speed(camera, increase=True):
    original_speed = camera.speed
    if increase == False:
        original_speed *= -1
    def closure(*_):
        camera.speed += original_speed
    return closure

# Atribuição dos eventos de movimentação da câmera
window.add_key_callback("W", camera.move_forward)
window.add_key_callback("D", camera.move_right)
window.add_key_callback("S", camera.move_backward)
window.add_key_callback("A", camera.move_left)
window.add_cursor_callback(camera.move_front)
window.add_key_callback("Q", modify_camera_speed(camera, False))
window.add_key_callback("E", modify_camera_speed(camera, True))

# Transformações

In [27]:
# Atributos de controle dos modelos
light_angle = 0.0
girl_angle = 0.0

# Para transformar a luz
def transform_light():
    global light_angle 
    light_angle += 1e-2
    if light_angle > 2.0 * math.pi:
        light_angle = 0.0
    tx = math.cos(light_angle) * 10
    ty = math.sin(light_angle) * 10
    tz = 3.0
    session.move_light(tx, ty, tz)

# Para transformar a garota
# ACHISMO DO TULIP: talvez seja bacana bindar pra uma tecla só, mas tbm n sei
def transform_girl():
    global girl_angle 
    girl_angle += 1e-5
    if girl_angle > 2.0 * math.pi:
        girl_angle = 0.0
    girl.set_model(yrotate(girl_angle, girl.model))

# Para transformar o penguin
# Dxr o penguin se mexendo, círculos talvez ? preciso ver dps
# Desconsidere os valores e situação dessa função por enquanto
def transform_penguin():
    penguin.set_model(translate(0.2, 0.01, 0.01, penguin.model))
    penguin.set_model(xrotate(1e-1, penguin.model))

# Execução

In [28]:
# Processamento dos buffers
session.process_buffers()

# Programa principal
for _ in window.display():

    # Limpeza da janela
    session.clear()

    # Posicionamento juntamente com a Lua, nossa fonte de luz neste caso
    session.move_light(10.0, 20.0, 2.0)

    # Transformações
    transform_light()
    transform_girl()

    # Renderização
    session.render()

    # Visualização
    session.view(camera.view)

    # Projeção
    session.projection(window.projection)